<a href="https://colab.research.google.com/github/InadiaI/FakeNews/blob/main/Copie_de_FakeNews_ELKHOURY_BIANNIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Subject :** Create an algorithm for classifying news are "fake" or "real".






_______________________________________________________

[1] Datasets from kaggle.com

https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset

https://www.kaggle.com/c/fake-news/data 

https://www.kaggle.com/hassanamin/textdb3

https://www.kaggle.com/snapcrack/all-the-news (only "real")

https://www.kaggle.com/mrisdal/fake-news (only fake)




1- You need datasets containing news with a label "fake" or "real". You can use datasets from kaggle.com [1] or any other dataset you can find (github, google colab, ...).


In [1]:
#We start by importing all libraries
import keras
!pip install kaggle
import json
import zipfile
import os
import pandas as pd
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# **Etape 1: Import et Formattage de nos données**
On importe la dataset de kaggle :

In [6]:
#kaggle token
os.environ['KAGGLE_USERNAME'] = "nadiaelkhoury"
os.environ['KAGGLE_KEY'] = "0dda7b45f140e015af3441566c4be78d"

#mkdir for a clean working environment
if not os.path.exists("/fakeNews/"):
    os.makedirs("/fakeNews")
os.chdir('/fakeNews')

#dl
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

zip = zipfile.ZipFile("fake-and-real-news-dataset.zip",'r')
zip.extractall()

for file in os.listdir():
  print(file)



fake-and-real-news-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Fake.csv
fake-and-real-news-dataset.zip
True.csv


On importe les différentes datasets True et Fake, on ajoute une colonne de labels pour pouvoir différencier les fake news des vrais ( 1 : true, à: fake) On va ensuite split le texte pour séparer chaque élément puis filtrer la ponctuation pour ne garder les mots.Enfin on concatène les deux bases de données en une seule qu'on appellera dataset.

In [58]:
#import puis traitement et filtrage des données
FILENAMES = {'True.csv':1, 'Fake.csv':0}
DATA_MAX = 1000 #max data number taken in each dataframes

dfs = []
for filename, label in FILENAMES.items():
  df = pd.read_csv(filename)
  df["y"] = label
  dfs.append(df[:DATA_MAX])


#concatenation
dataset = pd.concat(dfs, ignore_index=True)
dataset

,title,text,subject,date,y
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
1995,GOP Rep. Wants A $30k A Year Housing Allowanc...,"Not long ago, Rep. Jason Chaffetz (R-Utah), sa...",News,"June 28, 2017",0
1996,Brace Yourself For 74 Percent Higher Health C...,"According to a new report, health insurance pr...",News,"June 27, 2017",0
1997,Grandma Will Have To Pay More Than $20k A Yea...,The Congressional Budget Office released their...,News,"June 27, 2017",0
1998,The Absolutely Cringeworthy Moment Trump Trie...,Donald Trump was in a room full of Irish repor...,News,"June 27, 2017",0


formating text:
* tokenizing
* only alpha
* lowercase
* removal of useless auxiliaries

In [59]:
def my_tokenizer_func(string):
  words = np.asarray(word_tokenize(string))
  return np.core.defchararray.lower(words[np.where(np.core.defchararray.isalpha(words))])

def my_clean_func(table):
  stop_words = stopwords.words('english')
  words = []
  for i in table:
    if i not in stop_words:
      words.append(i)

  return words

dataset.text = dataset.text.apply(my_tokenizer_func)

#nettoyage des mots inutiles
from nltk.corpus import stopwords
nltk.download('stopwords')


for i in dataset.index:
    dataset.at[i, "text"] = my_clean_func(dataset.text[i]) 

dataset

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,title,text,subject,date,y
0,"As U.S. budget fight looms, Republicans flip t...","[washington, reuters, head, conservative, repu...",politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,"[washington, reuters, transgender, people, all...",politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,"[washington, reuters, special, counsel, invest...",politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,"[washington, reuters, trump, campaign, adviser...",politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,"[reuters, president, donald, trump, called, po...",politicsNews,"December 29, 2017",1
...,...,...,...,...,...
1995,GOP Rep. Wants A $30k A Year Housing Allowanc...,"[long, ago, jason, chaffetz, said, poor, peopl...",News,"June 28, 2017",0
1996,Brace Yourself For 74 Percent Higher Health C...,"[according, new, report, health, insurance, pr...",News,"June 27, 2017",0
1997,Grandma Will Have To Pay More Than $20k A Yea...,"[congressional, budget, office, released, repo...",News,"June 27, 2017",0
1998,The Absolutely Cringeworthy Moment Trump Trie...,"[donald, trump, room, full, irish, reporters, ...",News,"June 27, 2017",0


In [ ]:
train_dataset, other_datasets = train_test_split(dataset, train_size=0.4)
valid_dataset, test_dataset = train_test_split(other_datasets, train_size=0.5)

print(f"dataset shape : train:{train_dataset.shape}, valid:{valid_dataset.shape}, test{test_dataset.shape}")
print("\n Train dataset :")
train_dataset


dataset shape : train:(800, 5), valid:(600, 5), test(600, 5)

 Train dataset :


,title,text,subject,date,y
1711,It’s Not Over: Trump’s Tantrums Drive Senate ...,"[trumpcare, failed, in, the, most, hilarious, ...",News,"July 29, 2017",0
239,White House pressures Big Corn to meet on U.S....,"[reuters, president, donald, trump, s, adminis...",politicsNews,"December 8, 2017",1
779,Democrat Ralph Northam wins Virginia governor'...,"[washington, reuters, democrat, ralph, northam...",politicsNews,"November 8, 2017",1
58,Top Democrat says Trump firing of Mueller coul...,"[washington, reuters, the, top, democrat, on, ...",politicsNews,"December 20, 2017",1
669,Trump distances himself from remarks on Putin ...,"[hanoi, reuters, president, donald, trump, on,...",politicsNews,"November 12, 2017",1
...,...,...,...,...,...
1602,"Trump Golfs While Threatening Nuclear War, Th...","[any, sane, person, would, think, that, after,...",News,"August 9, 2017",0
1287,Obama’s Head Of Disaster Relief TRASHES Trump...,"[it, s, no, secret, that, donald, trump, s, re...",News,"September 30, 2017",0
587,U.S. nuclear general says would resist 'illega...,"[washington, reuters, the, top, nuclear, comma...",politicsNews,"November 18, 2017",1
1635,Breaking: MAJOR Action Taking Place At Trump ...,"[by, now, everyone, knows, that, donald, trump...",News,"August 8, 2017",0


## **Analysis of the dataset:** 
Comme on peut le remarquer notre dataset est formée des colonnes suivantes : un index, un titre, un texte, un sujet, une date, et on a rajouté une colonne y qui indique si la nouvelle est vrai ou fausse.
On l'a obtenue en mélangeant 2 datasets de vraies et fausses nouvelles.
Then you will process the data. Try to represent the data with Bag of Word
Apply a standard (not deep learning) machine learning algorithm.
Compute and analyze the performance.

try to apply the same algorithm to a different dataset,
use more complex representation of the data (Word2Vect, or anything else),
try to improve or change the ML algorithm (for exemple try to use deepLearning),
In each case, compare the performances obtained.


It is not expected that you try all of this, you are free to chose the representation of data that you want and the ML that you want. But be sure to understand what you are doing as you will need to present it.
